In [39]:
.engine.model.instrument.Instrument:([iId:`long$()] 
   cntTyp:`long$();state:`long$();faceValue:`long$();mkprice:`long$();
   plmts:`long$();plmtb:`long$();lotSize:`long$();tickSize:`long$();mnSize:`long$();
   mxSize:`long$();mnPrice:`long$();mxPrice:`long$();smul:`long$();
   bestBid:`long$();bestAsk:`long$();liqb:`boolean$();liqs:`boolean$());

In [34]:
.engine.model.inventory.Inventory:([aId:`long$();side:`long$()] iId:`.engine.model.instrument.Instrument$();
   ordQty:`long$();ordVal:`long$();ordLoss:`long$();amt:`long$();iw:`long$();mm:`long$();
   posVal:`long$();rpnl:`long$();avgPrice:`long$();execCost:`long$();upnl:`long$();lev:`long$());

In [50]:
.engine.model.risktier.Risktier:([rtId:`long$()] amt:`long$();mmtr:`float$();imr:`float$();lev:`long$());

In [51]:
.engine.model.feetier.Feetier:([ftId:`long$()] 
    bal:`long$();ref:`long$();vol:`long$();mkrfee:`float$();tkrfee:`float$();wdrawfee:`float$();
    dpstfee:`float$();wdlim:`long$());

In [139]:
.engine.model.account.Account:([aId:`long$()] 
  iId:`.engine.model.instrument.Instrument$();
  lng:`.engine.model.inventory.Inventory$();srt:`.engine.model.inventory.Inventory$();
  rt:`.engine.model.risktier.Risktier$();ft:`.engine.model.feetier.Feetier$();
  dep:`long$();wit:`long$();mrgTyp:`long$();avail:`long$();vol:`long$();bal:`long$());

In [1]:
\cd qenv

In [3]:
\cd ../../qenv

In [5]:
\l test.q

`aId`side`mm`upnl`ordQty`ordLoss`ordVal`amt`totEnt
(0 -1 0 0 0 0 0 10 10;0 1 0 0 0 0 0 10 10)
`fId`price`side`qty`reduce`ismaker`oId`aId`iId`time`ivId
((0;1000;1;100;0b;0b;0;0;0;2020.12.14T14:35:30.252;0 -1);(1;1000;-1;100;1b;1b;0;0;0;2020.12.14T14:35:30.252;0 1))
0 -1
0 1 
`.engine.model.inventory.Inventory!0 1
"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"36) .engine.logic.inventory.Fill (UNIT) :READY"
`Function for deriving the exec cost from the qty and the price
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                            
---------------------------------------------------

In [6]:
.util.testutils.makeDefaultsRecords  :{[tab;cl;vl] // TODO inter with actual cols
    r:.util.NullRowDict[tab];
    cvl:count[vl]; 
    :$[cvl>1;[rx:(cvl#enlist[r]);rx[cl]:flip[vl];:rx];[r[cl]:first[vl];:r]]};

In [7]:
.util.testutils.makeInstrument     :{[cl;vl]
    .util.testutils.makeDefaultsRecords[`.engine.model.instrument.Instrument;cl;vl]
    };

In [12]:
// Checks that the .engine.model.order.Order table matches the orders
// that are provided.
/  @param x (Order/List) The orders that are to be checked
/  @param y (Case) The case that the assertions belong to
/  @param z (List[String]) The params that are being checked 
.util.testutils.makeInventory     :{[cl;vl]
    .util.testutils.makeDefaultsRecords[`.engine.model.inventory.Inventory;cl;vl]
    };

In [9]:
.engine.model.instrument.Instrument,:.util.testutils.makeInstrument[`mkprice`tickSize`mnSize;enlist(1000;5;1)]

In [71]:
/  @param x (Order/List) The orders that are to be checked
/  @param y (Case) The case that the assertions belong to
/  @param z (List[String]) The params that are being checked 
.util.testutils.makeRisktier       :{[cl;vl]
    .util.testutils.makeDefaultsRecords[`.engine.model.risktier.Risktier;cl;vl]
    };

/  @param x (Order/List) The orders that are to be checked
/  @param y (Case) The case that the assertions belong to
/  @param z (List[String]) The params that are being checked 
.util.testutils.makeFeetier        :{[cl;vl]
    .util.testutils.makeDefaultsRecords[`.engine.model.feetier.Feetier;cl;vl]
    };

In [70]:
.engine.model.risktier.Risktier:([rtId:`long$()] amt:`long$();mmr:`float$();imr:`float$();lev:`long$());
.engine.model.risktier.Risktier,:.util.testutils.makeRisktier[`rtId`amt`mmr`imr`lev;(
        (0; 50000;       0.004;    0.008;    125);
        (1; 250000;      0.005;    0.01;     100);
        (2; 1000000;     0.01;     0.02;     50);
        (3; 5000000;     0.025;    0.05;     20);
        (4; 20000000;    0.05;     0.1;      10);
        (5; 50000000;    0.1;      0.20;     5);
        (6; 100000000;   0.125;    0.25;     4);
        (7; 200000000;   0.15;     0.333;    3);
        (8; 500000000;   0.25;     0.50;     2);
        (9; 500000000;   0.25;     1.0;      1))]; 

In [72]:
.engine.model.feetier.Feetier:([ftId:`long$()] 
    bal:`long$();ref:`long$();vol:`long$();mkrfee:`float$();tkrfee:`float$();wdrawfee:`float$();
    dpstfee:`float$();wdlim:`long$());
.engine.model.feetier.Feetier,:.util.testutils.makeFeetier[`ftId`vol`mkrfee`tkrfee`wdrawfee`dpstfee`wdlim;(
        (0; 50;      0.0006;    0.0006;    0f;  0f; 600);
        (1; 500;     0.00054;   0.0006;    0f;  0f; 600);
        (2; 1500;    0.00048;   0.0006;    0f;  0f; 600);
        (3; 4500;    0.00042;   0.0006;    0f;  0f; 600);
        (4; 10000;   0.00042;   0.00054;   0f;  0f; 600);
        (5; 20000;   0.00036;   0.00048;   0f;  0f; 600);
        (6; 40000;   0.00024;   0.00036;   0f;  0f; 600);
        (7; 80000;   0.00018;   0.000300;  0f;  0f; 600);
        (8; 150000;  0.00012;   0.00024;   0f;  0f; 600))];                

In [40]:
.engine.model.instrument.Instrument,:((!) . flip(
        (`iId                      ; 0);                                           
        (`cntTyp                   ; 0);
        (`state                    ; 0);
        (`faceValue                ; 0);
        (`mkprice                  ; 0);
        (`mxSize                   ; 0);
        (`mnPrice                  ; 0);
        (`smul                     ; 0);
        (`mxPrice                  ; 0) 
        ));

In [156]:
n:count[aIds];
dn:n*2;
.engine.model.inventory.Inventory,:ivn:flip[(!) . flip(
    (`aId              ; floor[til[dn]%2]); 
    (`iId              ; `.engine.model.instrument.Instrument$(dn#0)); 
    (`side             ; dn#(1 -1));                                            
    (`ordQty           ; dn#0);
    (`ordVal           ; dn#0);
    (`ordLoss          ; dn#0);
    (`amt              ; dn#10);
    (`iw               ; dn#0); // isolated wallet
    (`mm               ; dn#0);
    (`posVal           ; dn#9999);
    (`rpnl             ; dn#0);  
    (`avgPrice         ; dn#0);  
    (`execCost         ; dn#0);  
    (`upnl             ; dn#0);  
    (`lev              ; dn#0)  
    )];

In [140]:
// Checks that the .engine.model.order.Order table matches the orders
// that are provided.
/  @param x (Order/List) The orders that are to be checked
/  @param y (Case) The case that the assertions belong to
/  @param z (List[String]) The params that are being checked 
.util.testutils.makeAccount        :{[cl;vl]
    .util.testutils.makeDefaultsRecords[`.engine.model.account.Account;cl;vl]
    };
c:`aId`lng`srt`dep`wit`rt`ft`posTyp`mrgTyp`avail`bal;
acc:flip((!) . flip(
        (`aId              ; aIds); 
        (`iId              ; `.engine.model.instrument.Instrument$(n#0)); 
        (`lng              ; `.engine.model.inventory.Inventory$(flip(aIds;n#1)));
        (`srt              ; `.engine.model.inventory.Inventory$(flip(aIds;n#-1)));
        (`dep              ; n#0);  
        (`wit              ; n#0);  
        (`rt               ; n#`.engine.model.risktier.Risktier$0);  
        (`ft               ; n#`.engine.model.feetier.Feetier$0);  
        (`mrgTyp           ; n#0);  
        (`avail            ; n?10);  
        (`vol              ; n?10);  
        (`bal              ; n?10)  
        ));
.engine.model.account.Account,:acc;

In [25]:
update 
 mkprice:1000
 from `.engine.model.instrument.Instrument
 where iId=0;

In [79]:
update 
 upl:.engine.logic.contract.UnrealizedPnl[
     iId.cntTyp,
     iId.mkprice,
     iId.faceValue,
     iId.smul,
     amt,
     side,
     avgPrice]
    from `.engine.model.inventory.Inventory 
    where (iId=0) and (amt>0)

`.engine.model.inventory.Inventory


In [113]:
.engine.model.risktier.Risktier

rtId| amt       mmr   imr   lev
----| -------------------------
0   | 50000     0.004 0.008 125
1   | 250000    0.005 0.01  100
2   | 1000000   0.01  0.02  50 
3   | 5000000   0.025 0.05  20 
4   | 20000000  0.05  0.1   10 
5   | 50000000  0.1   0.2   5  
6   | 100000000 0.125 0.25  4  
7   | 200000000 0.15  0.333 3  
8   | 500000000 0.25  0.5   2  
9   | 500000000 0.25  1     1  


In [125]:
a:20000;
l:10;


rtId| amt   mmr   imr   lev
----| ---------------------
0   | 50000 0.004 0.008 125


In [142]:
.engine.logic.account.GetFeetier:{[avol]
   ft:select[1;<vol] from .engine.model.feetier.Feetier where (vol>a) or ((i=0) and (vol>a));
   `.engine.model.feetier.Feetier$((0!ft)`ftId)
   };

.engine.logic.account.GetRisktier:{[ivnamt;ivlev]
   rt:select[1;<amt] from .engine.model.risktier.Risktier where (amt>a) or ((i=0) and (amt>a));
   `.engine.model.risktier.Risktier$((0!rt)`rtId)
   };

In [128]:
.engine.logic.account.GetAvailable:{[bal;mm;upnl;oqty;oloss]
      bal-(mm+upnl)+(oqty-oloss)
    };

In [143]:
.engine.model.account.Account

aId| iId lng srt rt ft dep wit mrgTyp avail vol bal
---| ----------------------------------------------
0  | 0   0   1   0  0  0   0   0      9     0   9  
1  | 0   2   3   0  0  0   0   0      9     3   8  


In [145]:
.engine.model.inventory.Inventory

aId side| iId ordQty ordVal ordLoss amt iw mm posVal rpnl avgPrice execCost upnl lev upl
--------| ------------------------------------------------------------------------------
0   1   | 0   0      0      0       0   0  0  0      0    0        0        0    0      
0   -1  | 0   0      0      0       0   0  0  0      0    0        0        0    0      
1   1   | 0   0      0      0       0   0  0  0      0    0        0        0    0      
1   -1  | 0   0      0      0       0   0  0  0      0    0        0        0    0      


In [149]:
update
  ft:.engine.logic.account.GetFeetier'[vol],
  rt:.engine.logic.account.GetRisktier'[lng.amt+srt.amt;bal%(lng.amt+srt.amt)],
  avail:.engine.logic.account.GetAvailable'[
  bal;lng.mm+srt.mm;lng.upnl+srt.upnl;
  lng.ordQty+srt.ordQty;lng.ordLoss+srt.ordLoss]
      from `.engine.model.account.Account 
      where (iId=0) and ((lng.amt>0) or (srt.amt>0))

`.engine.model.account.Account


In [157]:
select
    rpnl:rpnl+(posVal * 0.001 * side) // TODO roundup
    from .engine.model.inventory.Inventory
    where (iId=0) and (amt>0)

rpnl  
------
9.999 
-9.999
9.999 
-9.999


In [ ]:
iv:.engine.model.inventory.Get[enlist(<;`amt;0)];
fnd:0!select 
        amtInMarket: sum[amt],
        fundingCost:((min[(fundingrate;0)]*(amt*side)) + (max[(fundingrate;0)]*(amt*side)))
        by aId from enlist iv;  

a:.engine.model.account.Get[fnd`aId];
a:.engine.logic.account.Remargin[i;a];

.engine.model.account.Update a;
.engine.model.inventory.Update iv;
.engine.Emit[`account;t;a];
.engine.Emit[`inventory;t;iv];

In [ ]:
.engine.logic.instrument.Settlement:{
                update 
                    bal:bal+(lng.rpnl+srt.rpnl)
                    from `.engine.model.account.Account;

                update
                rpnl:0
                from `.engine.model.inventory.Inventory;

                .engine.Emit[`account;t;a];
                .engine.Emit[`inventory;t;iv];
                .engine.Emit[`settlement;t;x];
	};

In [7]:
\l test.q

`aId`side`mm`upnl`ordQty`ordLoss`ordVal`amt`totEnt
(0 -1 0 0 0 0 0 10 10;0 1 0 0 0 0 0 10 10)
`fId`price`side`qty`reduce`ismaker`oId`aId`iId`time`ivId
((0;1000;1;100;0b;0b;0;0;0;2020.12.14T14:36:11.963;0 -1);(1;1000;-1;100;1b;1b;0;0;0;2020.12.14T14:36:11.963;0 1))
0 -1
0 1 
`.engine.model.inventory.Inventory!0 1
"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"36) .engine.logic.inventory.Fill (UNIT) :READY"
`Function for deriving the exec cost from the qty and the price
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                            
---------------------------------------------------

In [8]:
.bam.iv

{[cl;vl]
    //if[null cl;cl:key .fill.r]; // TODO check
    x:flip .model.Model[.engine.model.inventory.r;cl;vl];
    / x[`iId]:`.engine.model.instrument.Instrument$x[`iId];
    x
    }[+`fId`time`price`qty`reduce`ismaker`side`iId`oId`ivId`aId`kind`avgPrice`execCost`ordLoss`ordVal`ordQty`amt`totEnt`upnl`rpnl!(0 1;2020.12.14T14:36:11.963 2020.12.14T14:36:11.963;1000 1000;100 100;01b;01b;1 -1;`.engine.model.instrument.Instrument$0 0;`.engine.model.order.Order$0 0;`.engine.model.inventory.Inventory!0 1;`.engine.model.account.Account$0 0;`inventory`inventory;0 0;10000000 10000000;0 0;-100000000 -100000000;-100 -100;110 110;210 210;0 0;0 0)]


In [159]:
update amt:sums[] by aId from .engine.model.inventory.Inventory

aId side| iId ordQty ordVal ordLoss amt iw mm posVal rpnl avgPrice execCost upnl lev upl
--------| ------------------------------------------------------------------------------
0   1   | 0   0      0      0       15  0  0  9999   0    0        0        0    0      
0   -1  | 0   0      0      0       15  0  0  9999   0    0        0        0    0      
1   1   | 0   0      0      0       15  0  0  9999   0    0        0        0    0      
1   -1  | 0   0      0      0       15  0  0  9999   0    0        0        0    0      


In [161]:
iv:?[;();0b;(
    (`.engine.logic.contract.AvgPrice);
    (`.engine.logic.contract.ExecCost);
    (`.engine.logic.contract.Loss);
    (`.engine.logic.contract.Value);
    (`ordLoss);
    (`ordQty);
    (`ordVal);
    (`amt);
    (sum;`ivId.totalEntry;(max;`dlt;0))
    )];



[0;31mtype[0m: [0;31mtype[0m

In [176]:
?[flip .bam.x;();0b;(
     (`.engine.logic.contract.UnrealizedPnl;`iId.cntTyp;`iId.mkprice;`iId.faceValue;`iId.smul;`amt;`side;`avgPrice);
     (`.engine.logic.contract.RealizedPnl;`iId.cntTyp;`qty;`price;`side;`ivId.avgPrice;`iId.faceValue;`iId.smul)
     )]

[0;31mtype[0m: [0;31mtype[0m

In [176]:
select iId.cntTyp from .bam.x

cntTyp
------
0     
0     


In [170]:
first .bam.x

fId    | 0
price  | 1000
qty    | 100
reduce | 0b
ismaker| 0b
side   | 1
iId    | `.engine.model.account.Account$0
oId    | 0
ivId   | `.engine.model.inventory.Inventory!1
aId    | `.engine.model.account.Account$0
